In [1]:
!pip install -q beautifulsoup4

In [2]:
import sys

sys.path.append('..')

In [3]:
import os
import yaml
import requests
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
from pymongo import MongoClient
from utils import optimize_dtypes

In [4]:
BASE_PATH = '../../'

CONFIG_DIR = os.path.join(BASE_PATH, 'config')
STORAGE_DIR = os.path.join(BASE_PATH, 'storage')

config = yaml.load(open(os.path.join(CONFIG_DIR, 'env.yml')),
                   Loader=yaml.FullLoader)

In [5]:
model = 'devices'
model_storage = os.path.join(STORAGE_DIR, model)

client = MongoClient('mongodb://{}:{}@{}:{}'.format(config['MONGO_USERNAME'], config['MONGO_PASSWORD'],
                                                    config['MONGO_HOST'], config['MONGO_PORT']))
metadata_db = client[config['MONGO_DATABASE']]

model_metadata = metadata_db[model]

In [6]:
mappings = model_metadata.find({}, {'mappings': 1}).next()['mappings']

In [7]:
df = pd.read_parquet(os.path.join(model_storage, 'devices.0.parquet'))

In [8]:
df = optimize_dtypes(df, mappings)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307141 entries, 0 to 307140
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   id            307141 non-null  uint32        
 1   model         307141 non-null  category      
 2   manufacturer  307141 non-null  category      
 3   brand         307141 non-null  category      
 4   os_version    307141 non-null  category      
 5   is_root       307141 non-null  bool          
 6   created_at    307141 non-null  datetime64[ns]
dtypes: bool(1), category(4), datetime64[ns](1), uint32(1)
memory usage: 6.8 MB


In [10]:
page = requests.get('https://www.gsmarena.com/res.php3?sSearch=xiaomi+redmi+5')

In [12]:
soup = BeautifulSoup(page.content, 'html.parser')